In [ ]:
#from metrics import recall_k, ndcg_k, repeat_score_item, repeat_score_user

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
! pip install wandb

In [ ]:
import wandb
wandb.login(key='4da15e357372a40c1c71a28c87306f19bf380d80')

In [ ]:
def wandb_start(config, run_name):
    wandb.init(project="NBR_instacart", config=config)
    wandb.run.name = run_name

In [ ]:
config = {
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import os
import time
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import json
import itertools
import sys
from scipy.sparse import csr_matrix, vstack
from scipy import sparse

In [ ]:
basket_count_min = 3
min_item_count = 5
history_len = 20

In [ ]:
path_train = '/content/drive/MyDrive/insta_baskets_30k/train_baskets_30k.csv'  # для файлов что выше сделались
path_val = '/content/drive/MyDrive/insta_baskets_30k/valid_baskets_30k.csv'
path_test = '/content/drive/MyDrive/insta_baskets_30k/test_baskets_30k.csv'

In [ ]:
dataset = PreDataset(path_train,path_val, path_test, dataset=dataset_name, history_len=history_len,basket_count_min=3, min_item_count = 5)

number of test users: 28749
items: 43728
filtered items: 30002


## Было сделано 3 модификации модели, которые описаны ниже

## Modification 1

### Creating a multi-hot vector for each user about user's all items consuming followed some linear blocks, which will replace user_embedding

In [ ]:
multihot_dict = {}
for user_id, items_list in zip(train_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()['user_id'],
                               train_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()['item_id']):
    l1 = sorted(list(set([dataset.item_id_mapper[item] for item in items_list if item in dataset.item_id_mapper])))
    multihot_dict[dataset.user_id_mapper[user_id]] = [1 if i in l1 else 0 for i in np.array(list(dataset.id_item_mapper.keys()))]
    print(len(list(multihot_dict.keys())))

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
history_len=20
item_embed_size=128
user_embed_size=32

h1 = 128
h2 = 128
h3 = 128
h4 = 128
h5 = 128
dataset_name = 'du'

In [ ]:
dataset = PreDataset(path_train,path_val, path_test, dataset=dataset_name, history_len=history_len,basket_count_min=3, min_item_count = 5)

number of test users: 2483
items: 91764
filtered items: 36963


In [ ]:
train_baskets = pd.read_csv(path_train)
test_baskets = pd.read_csv(path_test)
valid_baskets = pd.read_csv(path_val)

In [ ]:
dataset

In [ ]:
test_baskets.shape, valid_baskets.shape, train_baskets.shape

((157182, 10), (158033, 10), (4594693, 10))

In [ ]:
batch_size = 4096

train_dataset = RCNDataset(dataset, mode='train')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = RCNDataset(dataset, mode='valid')
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = RCNDataset(dataset, mode='test')
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Data allready in use


In [ ]:
len(train_loader.dataset), len(val_loader.dataset), len(test_loader.dataset)

(50176675, 643538, 658898)

In [ ]:
len(train_loader.dataset), len(val_loader.dataset), len(test_loader.dataset)

(25733472, 929152, 933822)

### Block part with linear layer for running on multi-hot vector of each user

In [ ]:
class Block(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.dense = nn.Linear(dim_in, dim_out)
        self.dropout = nn.Dropout(0.2)
        #self.norm = nn.BatchNorm2d(dim_out)
        self.norm = nn.LayerNorm(dim_out, elementwise_affine = False)
        #self.dropout = nn.Dropout(0.2)
        self.act = nn.Tanh()
        
    def forward(self, x):
        return self.act(self.norm(self.dropout(self.dense(x))))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class ReCaNet(nn.Module):
    def __init__(self, num_items, item_embed_size, num_users, user_embed_size, history_len, h1, h2, h3, h4, h5):
        super(ReCaNet, self).__init__()
        self.num_items = num_items
        self.item_embed_size = item_embed_size
        self.num_users = num_users
        self.user_embed_size = user_embed_size
        self.history_len = history_len
        self.h1 = h1
        self.h2 = h2
        self.h3 = h3
        self.h4 = h4
        self.h5 = h5

        self.item_embedding = nn.Embedding(self.num_items, self.item_embed_size)
        self.user_embedding = nn.Embedding(self.num_users, self.user_embed_size)
        
        #self.fc1 = nn.Linear(self.item_embed_size + 36963, self.h1)
            #self.item_embed_size + self.user_embed_size, self.h1)
        self.fc1 = nn.Linear(self.item_embed_size + + self.user_embed_size, self.h1)
        self.fc2 = nn.Linear(self.h1+1, self.h2)
        self.lstm1 = nn.LSTM(self.h2, self.h3, batch_first=True, num_layers=2)
        #self.lstm2 = nn.LSTM(self.h3, self.h4, batch_first=True)
        self.fc5 = nn.Linear(self.h4, self.h5)
        self.fc6 = nn.Linear(self.h4, self.h5)
        self.fc7 = nn.Linear(self.h5, 1)
        self.layers = nn.Sequential(
               Block(36963, 2048), 
               Block(2048, 512), 
               Block(512, 128)
        )
        # self.encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=16, batch_first=True)
        # self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=8)

    def forward(self, input1, input2, input4):
        x1 = self.item_embedding(input1.to(torch.int64))
        x1 = x1.view(-1, self.item_embed_size) #1x128
       
        #x2 = self.user_embedding(input2.to(torch.int64))
        x2 = torch.tensor(np.array([multihot_dict[i.item()] for i in input2.to(torch.int64)])).long().to(torch.float32).to(device)
        x2 = x2.view(-1, 36963)
                     #self.user_embed_size) #1x32
        #x2 = x2.view(-1, self.user_embed_size)
        x2 = self.layers(x2)
        # 1x5
        x4 = input4 # 1x5
        x4 = x4.unsqueeze(2)
        
        conc1 = torch.cat((x1, x2), axis=1)
        x11 = F.relu(self.fc1(conc1)) # 1*128

        x12 = x11.unsqueeze(1).repeat(1, self.history_len, 1)
        
        conc2 = torch.cat((x12, x4), 2)#.transpose(-2,-1)
        x14 = F.relu(self.fc2(conc2))
        #valid_his = (input4 > 0).long()

        x21, (hx, cx) = self.lstm1(x14)             #Прежняя часть с LSTM
        x22 = hx[1]                                 #Прежняя часть с LSTM
       
        #x22 = self.transformer_encoder(x14).mean(dim=1)
        x = F.relu(self.fc5(x22))
        x = F.relu(self.fc6(x))
      
        output = self.fc7(x).view(-1) 
        output = torch.sigmoid(output)
        return output

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_new = ReCaNet(num_items=dataset.num_items, item_embed_size=item_embed_size, num_users=dataset.num_users, 
                user_embed_size=user_embed_size, history_len = history_len, h1 = h1,h2 = h2, h3 = h3,
                h4 = h4, h5 = h5).to(device)

#path_best = '/kaggle/input/weights/epoch_5_recanet_model_dunnhumby.pth'
#model_new.load_state_dict(torch.load(path_best))

In [ ]:
wandb_start(config, 'recanet_instacart')

In [ ]:
wandb.watch(model_new)

[]

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

In [ ]:
best_model = model_new

import torch.nn.functional as F # 22min
test_baskets = pd.read_csv(path_test) 
#path_test
#pd.read_csv(path_test)
user_test_baskets_df = test_baskets.groupby('user_id')['item_id'].apply(list).reset_index()
user_test_baskets_dict = dict(zip( user_test_baskets_df['user_id'],user_test_baskets_df['item_id']))

user_predictions = predict(best_model, test_loader)
final_users = set(dataset.test_users).intersection(set(list(user_test_baskets_dict.keys())))
print('predictions ready',len(user_predictions))
print('number of final test users:',len(final_users))
for k in [5,10,20,'B']:
    print(k)
    recall_scores = {}
    ndcg_scores = {}
    #zero = 0
    for user in final_users:

        top_items = []
        if user in user_predictions:
            top_items = user_predictions[user]
        else:
            zero+=1

        if k == 'B':
            recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
            ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
        else:
            recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,k)
            ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,k)
    #print(zero)
    print('recall:',np.mean(list(recall_scores.values())))
    print('ndcg:',np.mean(list(ndcg_scores.values())))

  0%|          | 0/228 [00:00<?, ?it/s]

predictions ready 28749
number of final test users: 14366
5
recall: 0.253004308674264
ndcg: 0.4227910320972693
10
recall: 0.3581563042608666
ndcg: 0.3515862669553688
20
recall: 0.46451642003850374
ndcg: 0.2726473842689201
B
recall: 0.329087256964059
ndcg: 0.37177391142322236


## Modification 2

### Replacing 2 sequential LSTM layers with the TransformerEncoder with 4 layers and 4 heads, with embedding dimension of 128

self.lstm1 = nn.LSTM(self.h2, self.h3, batch_first=True, num_layers=2)

   x21, (hx, cx) = self.lstm1(x14)             
   x22 = hx[1]     --------------->>>>>>>>>>>>>>>>>>>

   --------------->>>>>>>>>>>>>>>>>>>

self.encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=4, batch_first=True)
self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=4)
x22 = self.transformer_encoder(x14).mean(dim=1)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class ReCaNet(nn.Module):
    def __init__(self, num_items, item_embed_size, num_users, user_embed_size, history_len, h1, h2, h3, h4, h5):
        super(ReCaNet, self).__init__()
        self.num_items = num_items
        self.item_embed_size = item_embed_size
        self.num_users = num_users
        self.user_embed_size = user_embed_size
        self.history_len = history_len
        self.h1 = h1
        self.h2 = h2
        self.h3 = h3
        self.h4 = h4
        self.h5 = h5

        self.item_embedding = nn.Embedding(self.num_items, self.item_embed_size)
        self.user_embedding = nn.Embedding(self.num_users, self.user_embed_size)
        
        #self.fc1 = nn.Linear(self.item_embed_size + 36963, self.h1)
            #self.item_embed_size + self.user_embed_size, self.h1)
        self.fc1 = nn.Linear(self.item_embed_size + + self.user_embed_size, self.h1)
        self.fc2 = nn.Linear(self.h1+1, self.h2)
        self.lstm1 = nn.LSTM(self.h2, self.h3, batch_first=True, num_layers=2)
        #self.lstm2 = nn.LSTM(self.h3, self.h4, batch_first=True)
        self.fc5 = nn.Linear(self.h4, self.h5)
        self.fc6 = nn.Linear(self.h4, self.h5)
        self.fc7 = nn.Linear(self.h5, 1)
        # self.layers = nn.Sequential(
        #        Block(36963, 2048), 
        #        Block(2048, 512), 
        #        Block(512, 128)
        # )
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=4, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=4)

    def forward(self, input1, input2, input4):
        x1 = self.item_embedding(input1.to(torch.int64))
        x1 = x1.view(-1, self.item_embed_size) #1x128
       
        x2 = self.user_embedding(input2.to(torch.int64))
        #x2 = torch.tensor(np.array([multihot_dict[i.item()] for i in input2.to(torch.int64)])).long().to(torch.float32).to(device)
        #x2 = x2.view(-1, 36963)
                     #self.user_embed_size) #1x32
        x2 = x2.view(-1, self.user_embed_size)
        #x2 = self.layers(x2)
        # 1x5
        x4 = input4 # 1x5
        x4 = x4.unsqueeze(2)
        
        conc1 = torch.cat((x1, x2), axis=1)
        x11 = F.relu(self.fc1(conc1)) # 1*128

        x12 = x11.unsqueeze(1).repeat(1, self.history_len, 1)
        
        conc2 = torch.cat((x12, x4), 2)#.transpose(-2,-1)
        x14 = F.relu(self.fc2(conc2))
        #valid_his = (input4 > 0).long()

        # x21, (hx, cx) = self.lstm1(x14)             #Прежняя часть с LSTM
        # x22 = hx[1]                                 #Прежняя часть с LSTM
       
        x22 = self.transformer_encoder(x14).mean(dim=1)
        x = F.relu(self.fc5(x22))
        x = F.relu(self.fc6(x))
      
        output = self.fc7(x).view(-1) 
        output = torch.sigmoid(output)
        return output

## Modification 3

### Predicting additional new explore items for user from global top-1000 popular items in all dataset

top_popular = self.train_baskets.item_id.value_counts().index.tolist()[:1000]


                for explore_item in top_popular:
                    if dataset.item_id_mapper[explore_item] not in train_items:
                        train_users.append(dataset.user_id_mapper[user])
                        train_items.append(dataset.item_id_mapper[explore_item])
                        #train_history.append(real_input_history[-dataset.history_len:])
                        train_history2.append([0 for i in range(20)])
                        #print(item, basket_items_dict[label_basket])
                        train_labels.append(float(explore_item in basket_items_dict[label_basket]))

In [ ]:
# preparing dataset for torch.dataset class
class PreDataset():
    def __init__(self, path_train,path_val, path_test, dataset, history_len, basket_count_min=0,\
                                                         min_item_count = 0):
        self.basket_count_min = basket_count_min
        self.min_item_count = min_item_count
        self.history_len = history_len
        
        self.train_baskets = pd.read_csv(path_train)
        self.valid_baskets = pd.read_csv(path_val)
        self.test_baskets = pd.read_csv(path_test)
        
        basket_per_user = self.train_baskets[['user_id','basket_id']].drop_duplicates() \
            .groupby('user_id').agg({'basket_id':'count'}).reset_index()
        self.test_users = basket_per_user[basket_per_user['basket_id'] >= self.basket_count_min]['user_id'].tolist()
    
        print("number of test users:", len(self.test_users))
        
        self.model_name = 'data/dunnhumby_cj/'+dataset+ '_recanet'
        self.dataset = dataset
        self.all_items = self.train_baskets[['item_id']].drop_duplicates()['item_id'].tolist()
        self.all_users = self.train_baskets[['user_id']].drop_duplicates()['user_id'].tolist()
        self.num_items = len(self.all_items) +1
        self.num_users = len(self.all_users) +1

        print("items:", self.num_items)
        item_counts = self.train_baskets.groupby(['item_id']).size().to_frame(name = 'item_count').reset_index()
        item_counts = item_counts[item_counts['item_count']>= min_item_count]
        item_counts_dict = dict(zip(item_counts['item_id'], item_counts['item_count']))
        print("filtered items:", len(item_counts_dict))
        self.num_items = len(item_counts_dict) +1
        self.item_id_mapper = {} 
        self.id_item_mapper = {} 
        self.user_id_mapper = {} 
        self.id_user_mapper = {} 

        counter = 0
        for i in range(len(self.all_items)):
            if self.all_items[i] in item_counts_dict:
                self.item_id_mapper[self.all_items[i]] = counter+1
                self.id_item_mapper[counter+1] = self.all_items[i]
                counter+=1
        for i in range(len(self.all_users)):
            self.user_id_mapper[self.all_users[i]] = i+1
            self.id_user_mapper[i+1] = self.all_users[i]
            
            
    def create_train_data(self):
        if os.path.isfile('/content/drive/MyDrive/tafeng_data/20_test_history22.npy'):
        #if os.path.isfile('/kaggle/input/npy-files/dunnhumby_cj_recanet_20_test_history2.npy'):
        #if os.path.isfile(self.model_name +'_' + str(self.history_len) + '_train_users.npy'):
            print('Data allready in use')
            train_users = np.load('/content/drive/MyDrive/tafeng_data/20' + '_train_users.npy')
            train_items = np.load('/content/drive/MyDrive/tafeng_data/20' + '_train_items.npy')
            #train_history = np.load(self.model_name +'_' + str(self.history_len) + '_train_history.npy')
            train_history2 = np.load('/content/drive/MyDrive/tafeng_data/20' + '_train_history2.npy')
            train_labels = np.load('/content/drive/MyDrive/tafeng_data/20'+ '_train_labels.npy')
            return train_items,train_users, train_history2, train_labels
        row_counts = 0 
        basket_items = self.train_baskets.groupby(['basket_id'])['item_id'].apply(list).reset_index()
        basket_items_dict = dict(zip(basket_items['basket_id'],basket_items['item_id']))
        basket_items_dict['null'] = [] 

        user_baskets = self.train_baskets[['user_id','date','basket_id']].drop_duplicates().\
            sort_values(['user_id','date'],ascending=True).groupby(['user_id'])['basket_id'].apply(list).reset_index()

        user_baskets_dict = dict(zip(user_baskets['user_id'], user_baskets['basket_id']))
        top_popular = self.train_baskets.item_id.value_counts().index.tolist()[:1000]


        train_users = []
        train_items = []
        train_history = []
        train_history2 = []
        train_labels = []
        print('num users:', len(self.test_users))

        for c,user in tqdm(enumerate(self.test_users)):
            if c % 1000 ==1:
                print(c , 'user passed')

            baskets = user_baskets_dict[user]
            item_seq = {}
            for i, basket in enumerate(baskets):
                for item in basket_items_dict[basket]:
                    if item not in self.item_id_mapper:
                        continue
                    if item not in item_seq:
                        item_seq[item] = []
                    item_seq[item].append(i)


            for i in range(max(0,len(baskets)-50), len(baskets)):
                label_basket = baskets[i]
                all_history_baskets = baskets[:i]
                items = []
                for basket in all_history_baskets:
                    for item in basket_items_dict[basket]:
                        items.append(item)
                items = list(set(items))
                for item in items:
                    if item not in self.item_id_mapper:
                        continue
                    index = np.argmax(np.array(item_seq[item])>=i)
                    if np.max(np.array(item_seq[item])) < i:
                        index = len(item_seq[item])
                    input_history = item_seq[item][:index].copy()
                    if len(input_history) ==0:
                        continue
                    if len(input_history) ==1 and input_history[0]==-1:
                        continue
                    while len(input_history) < self.history_len:
                        input_history.insert(0,-1)
                    real_input_history = []
                    for x in input_history:
                        if x == -1:
                            real_input_history.append(0)
                        else:
                            real_input_history.append(i-x)
                    real_input_history2 = []
                    for j,x in enumerate(input_history[:-1]):
                        if x == -1:
                            real_input_history2.append(0)
                        else:
                            real_input_history2.append(input_history[j+1]-input_history[j])
                    real_input_history2.append(i-input_history[-1])
                    train_users.append(self.user_id_mapper[user])
                    train_items.append(self.item_id_mapper[item])
                    train_history.append(real_input_history[-self.history_len:])
                    train_history2.append(real_input_history2[-self.history_len:])
                    #print(item, basket_items_dict[label_basket])
                    train_labels.append(float(item in basket_items_dict[label_basket]))
                for explore_item in top_popular:
                    if dataset.item_id_mapper[explore_item] not in train_items:
                        train_users.append(dataset.user_id_mapper[user])
                        train_items.append(dataset.item_id_mapper[explore_item])
                        #train_history.append(real_input_history[-dataset.history_len:])
                        train_history2.append([0 for i in range(20)])
                        #print(item, basket_items_dict[label_basket])
                        train_labels.append(float(explore_item in basket_items_dict[label_basket]))

                    row_counts +=1
            print(row_counts)

        train_items = np.array(train_items)
        train_users = np.array(train_users)
        #train_history = np.array(train_history)
        train_history2 = np.array(train_history2)
        train_labels = np.array(train_labels)
        random_indices = np.random.choice(range(len(train_items)), len(train_items),replace=False)
        train_items = train_items[random_indices]
        train_users = train_users[random_indices]
        #train_history = train_history[random_indices]
        train_history2 = train_history2[random_indices]
        train_labels = train_labels[random_indices]        

        np.save('/content/drive/MyDrive/tafeng_data/' + str(self.history_len) + '_train_items.npy',train_items)
        np.save('/content/drive/MyDrive/tafeng_data/' + str(self.history_len) + '_train_users.npy',train_users)
      
        np.save('/content/drive/MyDrive/tafeng_data/' + str(self.history_len) + '_train_history2.npy',train_history2)
        np.save('/content/drive/MyDrive/tafeng_data/' + str(self.history_len) + '_train_labels.npy',train_labels)

        return train_items,train_users, train_history2 , train_labels
    
    
    def create_test_data(self,test_data='test'):
        if os.path.isfile('/content/drive/MyDrive/tafeng_data/20_test_history22.npy'):
        #if os.path.isfile('/kaggle/input/npy-files/dunnhumby_cj_recanet_20_test_history2.npy'):
        #if os.path.isfile(self.model_name +'_' + str(self.history_len)+ '_'+test_data+'_users.npy'):
            test_users = np.load('/content/drive/MyDrive/tafeng_data/20' + '_'+test_data+'_users.npy')
            test_items = np.load('/content/drive/MyDrive/tafeng_data/20' + '_'+test_data+'_items.npy')
            #test_history = np.load(self.model_name +'_' + str(self.history_len) + '_'+test_data+'_history.npy')
            test_history2 = np.load('/content/drive/MyDrive/tafeng_data/20' + '_'+test_data+'_history2.npy')
            test_labels = np.load('/content/drive/MyDrive/tafeng_data/20' + '_'+test_data+'_labels.npy')
            return test_items,test_users,test_history2, test_labels

        train_basket_items = self.train_baskets.groupby(['basket_id'])['item_id'].apply(list).reset_index()
        train_basket_items_dict = dict(zip(train_basket_items['basket_id'],train_basket_items['item_id']))

        train_user_baskets = self.train_baskets[['user_id','date','basket_id']].drop_duplicates(). \
            sort_values(['user_id','date'],ascending=True).groupby(['user_id'])['basket_id'].apply(list).reset_index()
        train_user_baskets_dict = dict(zip(train_user_baskets['user_id'],train_user_baskets['basket_id']))

        train_user_items = self.train_baskets[['user_id','item_id']].drop_duplicates().groupby(['user_id'])['item_id'] \
            .apply(list).reset_index()
        train_user_items_dict = dict(zip(train_user_items['user_id'],train_user_items['item_id']))

        test_user_items = None
        if test_data == 'test':
            test_user_items = self.test_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()
        else:
            test_user_items = self.valid_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()
        test_user_items_dict = dict(zip(test_user_items['user_id'],test_user_items['item_id']))

        top_popular = self.train_baskets.item_id.value_counts().index.tolist()[:1000]

        test_users = []
        test_items = []
        test_history = []
        test_history2 = []
        test_labels = []

        train_basket_items_dict['null'] = []
        for c,user in tqdm(enumerate(test_user_items_dict)):
            if user not in train_user_baskets_dict:
                continue
            if c % 100 ==1:
                print(c , 'user passed')
                #break

            baskets = train_user_baskets_dict[user]
            item_seq = {}
            for i, basket in enumerate(baskets):
                for item in train_basket_items_dict[basket]:
                    if item not in self.item_id_mapper:
                        continue
                    if item not in item_seq:
                        item_seq[item] = []
                    item_seq[item].append(i)


            label_items = test_user_items_dict[user]

            items = list(set(train_user_items_dict[user]))

            #print(len(history_baskets))
            for item in items:#train_user_items_dict[user]:
                if item not in self.item_id_mapper:
                    continue
                input_history = item_seq[item][-self.history_len:]
                if len(input_history) ==0:
                    continue
                if len(input_history) ==1 and input_history[0]==-1:
                    continue
                while len(input_history) < self.history_len:
                    input_history.insert(0,-1)

                real_input_history2 = []
                for j,x in enumerate(input_history[:-1]):
                    if x == -1:
                        real_input_history2.append(0)
                    else:
                        real_input_history2.append(input_history[j+1]-input_history[j])
                real_input_history2.append(len(baskets)-input_history[-1])
                test_users.append(self.user_id_mapper[user])
                test_items.append(self.item_id_mapper[item])
                #test_history.append(real_input_history)
                test_history2.append(real_input_history2)
                test_labels.append(float(item in label_items))
            for explore_item in top_popular:
                if dataset.item_id_mapper[explore_item] not in test_items:
                    test_users.append(self.user_id_mapper[user])
                    test_items.append(self.item_id_mapper[explore_item])
                    test_history2.append([0 for i in range(20)])
                    #print(item, basket_items_dict[label_basket])
                    test_labels.append(float(explore_item in label_items))

        test_items = np.array(test_items)
        test_users = np.array(test_users)
        #test_history = np.array(test_history)
        test_history2 = np.array(test_history2)
        test_labels = np.array(test_labels)

        np.save('/content/drive/MyDrive/tafeng_data/20' + '_'+test_data+'_items.npy',test_items)
        np.save('/content/drive/MyDrive/tafeng_data/'+ str(self.history_len) + '_'+test_data+'_users.npy',test_users)
       
        np.save('/content/drive/MyDrive/tafeng_data/'+ str(self.history_len) + '_'+test_data+'_history2.npy',test_history2)
        #sparse.save_npz('Downloads/dunnhumby/'+ str(self.history_len) + '_'+test_data+'_history2.npz', test_history2)
        np.save('/content/drive/MyDrive/tafeng_data/'+ str(self.history_len) + '_'+test_data+'_labels.npy',test_labels)

        return test_items, test_users, test_history2, test_labels